In [1]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 58.1 MB/s 


In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
spark = SparkSession.builder.getOrCreate()

In [4]:
spark

Installed the latest version of spark that is version 3

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Connecting to personal drive so as to access the uploaded dataset

In [6]:
df = spark.read.csv("/content/drive/MyDrive/Fraud.csv",inferSchema = True, header = True)

Reading the CSV dataset by creating an instance"df"

In [7]:
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



Checking the top 5 rows from our dataframe

In [8]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

This a large dataset so we are selecting some of the columns from the above dataset on which we will be perfoming out analysis.

In [76]:
df.count()

6362620

In [77]:
df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   8213|
|      0|6354407|
+-------+-------+



In [9]:
df = df.select("type","amount","oldbalanceOrg","newbalanceOrig", "isFraud")

First 5 rows form our customized dataset


In [10]:
df.show(5)

+--------+--------+-------------+--------------+-------+
|    type|  amount|oldbalanceOrg|newbalanceOrig|isFraud|
+--------+--------+-------------+--------------+-------+
| PAYMENT| 9839.64|     170136.0|     160296.36|      0|
| PAYMENT| 1864.28|      21249.0|      19384.72|      0|
|TRANSFER|   181.0|        181.0|           0.0|      1|
|CASH_OUT|   181.0|        181.0|           0.0|      1|
| PAYMENT|11668.14|      41554.0|      29885.86|      0|
+--------+--------+-------------+--------------+-------+
only showing top 5 rows



"**amount**", "**oldbalanceOrg**" and "**newbalanceorig**" are continuous coloumn and are found to be almost in same scale therefore we haven't done any preprocessing here at present


**Train/Test split** - We did the train test split at first so that there is no chance of data leakage in our analysis


We have splitted the dataset into 70:30 ratio for training and validating the model respectively. also we have splitted by taking seed equals to 42 so that everytime we run the file we will be working with the same splitted file, and there is no time series data we have used randomsplit to split the data into train and test


In [11]:
train,test = df.randomSplit([0.7,0.3], seed = 42)

In [12]:
train.show()

+-------+------+-------------+--------------+-------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+------+-------------+--------------+-------+
|CASH_IN|  1.42|   1270713.41|    1270714.83|      0|
|CASH_IN|  4.35|   4136277.22|    4136281.57|      0|
|CASH_IN|  4.71|      50198.0|      50202.71|      0|
|CASH_IN|  5.19|      18104.0|      18109.19|      0|
|CASH_IN|  5.44|          0.0|          5.44|      0|
|CASH_IN|  6.07|     400680.0|     400686.07|      0|
|CASH_IN|  8.27|   8428410.94|    8428419.21|      0|
|CASH_IN|  8.29|      20392.0|      20400.29|      0|
|CASH_IN|  8.44|      39384.0|      39392.44|      0|
|CASH_IN| 11.13|   4116439.74|    4116450.87|      0|
|CASH_IN| 12.18|     299322.0|     299334.18|      0|
|CASH_IN| 12.79|     601743.0|     601755.79|      0|
|CASH_IN| 13.86|   6868100.18|    6868114.04|      0|
|CASH_IN|  14.4|1.143460813E7| 1.143462253E7|      0|
|CASH_IN| 15.59| 1.64294897E7| 1.642950528E7|      0|
|CASH_IN|  16.3|2.140511936E

In [13]:
print(f"Train set length: {train.count()} records")
print(f"Test set length: {test.count()} records")

Train set length: 4454014 records
Test set length: 1908606 records


In [14]:
test.show(10)

+-------+------+-------------+--------------+-------+
|   type|amount|oldbalanceOrg|newbalanceOrig|isFraud|
+-------+------+-------------+--------------+-------+
|CASH_IN|  4.58|      94241.0|      94245.58|      0|
|CASH_IN|  5.66|   5061561.06|    5061566.72|      0|
|CASH_IN|   6.5|   1696433.45|    1696439.95|      0|
|CASH_IN|  6.76|      11322.0|      11328.76|      0|
|CASH_IN|  9.02|   2416260.59|    2416269.61|      0|
|CASH_IN|  9.04|      99971.0|      99980.04|      0|
|CASH_IN|  9.22|    7730148.9|    7730158.12|      0|
|CASH_IN|  13.2|     106204.0|      106217.2|      0|
|CASH_IN| 14.36|    613030.46|     613044.82|      0|
|CASH_IN| 14.54|    3347286.5|    3347301.03|      0|
+-------+------+-------------+--------------+-------+
only showing top 10 rows



**Dtypes** - In this dataset the column of type string has been treated as a categorical attricute. But sometimes it may happen that we treat a categorical attribute as a numerical one and viceversa, based on our requirements. This need to be carefuly indentified and carry on our preprocessing and feature extraction task accordingly.

In [15]:
train.dtypes

[('type', 'string'),
 ('amount', 'double'),
 ('oldbalanceOrg', 'double'),
 ('newbalanceOrig', 'double'),
 ('isFraud', 'int')]

Separating the categorical and numerical column accordingly

In [16]:
catCols = [x for (x, dataType) in train.dtypes if dataType == 'string']

In [17]:
numCols = [x for (x,dataType) in train.dtypes if ((dataType == "double") & (x != "isFraud"))]

In [18]:
print(catCols)

['type']


In [19]:
print(numCols)

['amount', 'oldbalanceOrg', 'newbalanceOrig']


**One Hot Encoding** - Feature Extraction

*  **Stringindexer :** Use to convert a single feature into an index   feature as it is mainly used to perform the task of ordering.

*  **One Hot Encoding:** converting the different categories of a categorical fetaure into numerical feature for ease of calculation without changing the original or raw dataset.


In [20]:
train.select("type").distinct().collect() #seeing all the unique/distinct value of the "type" column of train set

[Row(type='TRANSFER'),
 Row(type='CASH_IN'),
 Row(type='CASH_OUT'),
 Row(type='PAYMENT'),
 Row(type='DEBIT')]

In [21]:
test.select("type").distinct().collect() #seeing all the unique/distinct value of the "type" column of test set.

[Row(type='TRANSFER'),
 Row(type='CASH_IN'),
 Row(type='CASH_OUT'),
 Row(type='PAYMENT'),
 Row(type='DEBIT')]

In [22]:
train.groupBy("type").count().show()  #giving output as the total count of each feature in the type column of train set.

+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 372877|
| CASH_IN| 979432|
|CASH_OUT|1566920|
| PAYMENT|1505837|
|   DEBIT|  28948|
+--------+-------+



In [23]:
from pyspark.ml.feature import (OneHotEncoder,StringIndexer)

In [24]:
string_indexer = [StringIndexer(inputCol = x, outputCol= x+ "_StringIndexer", handleInvalid= "skip") for x in catCols]

In [25]:
string_indexer  #Giving a single stringindexer as output as because there is only one categorial attribute named "type".

[StringIndexer_d1c96cce86d8]

In [26]:
one_hot_encoder = [OneHotEncoder(inputCols=[f"{x}_StringIndexer" for x in catCols], outputCols=[f"{x}_OneHotEncoder" for x in catCols])]

In [27]:
one_hot_encoder #Giving a single OneHotEncoder as output as because there is only one categorial attribute named "type".

[OneHotEncoder_c8cae45a484b]

**Vector Assembler** - combines the values of input columns into a single vector

In [28]:
from pyspark.ml.feature import VectorAssembler

In [29]:
assemblerInput = [x for x in numCols]  # not doing any chnage for numerical columns as there are already in the same range
assemblerInput += [f"{x}_OneHotEncoder" for x in catCols]

In [30]:
assemblerInput #output given for all required columns into one 

['amount', 'oldbalanceOrg', 'newbalanceOrig', 'type_OneHotEncoder']

In [31]:
vector_assembler = VectorAssembler(inputCols= assemblerInput, outputCol = "VectorAssembler_features") #input features into a vector type

In [32]:
stages = []        #inorder to feed the pipeline that we will create we have build the stages to feed into the pipeline
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]

In [33]:
stages #three stages are shown

[StringIndexer_d1c96cce86d8,
 OneHotEncoder_c8cae45a484b,
 VectorAssembler_676d9dab00dc]

*   time given to see the time taken by the entire cell to execute.
*   set the stages into the pipeline
*   trained our model by fitting the prepared train data
*   based on the training data we transformed our test data to carry out further processes



In [34]:
%%time
from pyspark.ml import Pipeline  

pipeline = Pipeline().setStages(stages)
model = pipeline.fit(train)  

pp_df = model.transform(test) 


CPU times: user 184 ms, sys: 30.5 ms, total: 214 ms
Wall time: 25.4 s


In [35]:
pp_df.select("type","amount","oldbalanceOrg","newbalanceOrig","VectorAssembler_features", "isFraud").show(truncate = False)

+-------+------+-------------+--------------+--------------------------------------------------+-------+
|type   |amount|oldbalanceOrg|newbalanceOrig|VectorAssembler_features                          |isFraud|
+-------+------+-------------+--------------+--------------------------------------------------+-------+
|CASH_IN|4.58  |94241.0      |94245.58      |[4.58,94241.0,94245.58,0.0,0.0,1.0,0.0]           |0      |
|CASH_IN|5.66  |5061561.06   |5061566.72    |[5.66,5061561.06,5061566.72,0.0,0.0,1.0,0.0]      |0      |
|CASH_IN|6.5   |1696433.45   |1696439.95    |[6.5,1696433.45,1696439.95,0.0,0.0,1.0,0.0]       |0      |
|CASH_IN|6.76  |11322.0      |11328.76      |[6.76,11322.0,11328.76,0.0,0.0,1.0,0.0]           |0      |
|CASH_IN|9.02  |2416260.59   |2416269.61    |[9.02,2416260.59,2416269.61,0.0,0.0,1.0,0.0]      |0      |
|CASH_IN|9.04  |99971.0      |99980.04      |[9.04,99971.0,99980.04,0.0,0.0,1.0,0.0]           |0      |
|CASH_IN|9.22  |7730148.9    |7730158.12    |[9.22,7730

**Logistic regression** - Machine learning Model Building

In [36]:
from pyspark.ml.classification import LogisticRegression

In [37]:
data = pp_df.select(F.col("VectorAssembler_features").alias ("features"), F.col("isFraud").alias("label"))

In [38]:
data.show(5,truncate = False)

+--------------------------------------------+-----+
|features                                    |label|
+--------------------------------------------+-----+
|[4.58,94241.0,94245.58,0.0,0.0,1.0,0.0]     |0    |
|[5.66,5061561.06,5061566.72,0.0,0.0,1.0,0.0]|0    |
|[6.5,1696433.45,1696439.95,0.0,0.0,1.0,0.0] |0    |
|[6.76,11322.0,11328.76,0.0,0.0,1.0,0.0]     |0    |
|[9.02,2416260.59,2416269.61,0.0,0.0,1.0,0.0]|0    |
+--------------------------------------------+-----+
only showing top 5 rows



In [39]:
%%time
model = LogisticRegression().fit(data)

CPU times: user 510 ms, sys: 55.4 ms, total: 566 ms
Wall time: 1min 28s


In [40]:
model.summary.areaUnderROC #area under curve

0.9918411620092897

In [41]:
model.summary.pr.show() #showig precision and recall

+-------------------+-------------------+
|             recall|          precision|
+-------------------+-------------------+
|                0.0| 0.9195046439628483|
|0.35441527446300713| 0.9195046439628483|
|  0.469769291964996| 0.6683644595359366|
| 0.5338106603023071| 0.5231968810916179|
| 0.5731901352426412|0.42848647041332144|
| 0.6089896579156723|0.36794039894256186|
| 0.6340493237867939| 0.3214357733413995|
| 0.6634844868735084| 0.2897342365815529|
| 0.6897374701670644| 0.2645308924485126|
| 0.7108194112967383|0.24303005575955391|
| 0.7287191726332538|0.22475769844190896|
| 0.7458233890214797|0.20952061682869594|
| 0.7613365155131265| 0.1963681132656202|
| 0.7724741447891806|0.18416311047889997|
| 0.7852028639618138| 0.1740280349113991|
| 0.7975338106603023|  0.165142904208879|
| 0.8003182179793158|0.15548686244204019|
| 0.8027048528241846|0.14689183287232493|
| 0.8042959427207638| 0.1391029168959824|
| 0.8066825775656324|0.13225511934263728|
+-------------------+-------------

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


**Another way of doing the same project without using pipeline**

In [42]:
from pyspark.ml.feature import StringIndexer

In [43]:
type_indexer = StringIndexer(inputCol="type", outputCol="type_indexer").fit(df)
df = type_indexer.transform(df)

In [44]:
df.show(3,False)

+--------+-------+-------------+--------------+-------+------------+
|type    |amount |oldbalanceOrg|newbalanceOrig|isFraud|type_indexer|
+--------+-------+-------------+--------------+-------+------------+
|PAYMENT |9839.64|170136.0     |160296.36     |0      |1.0         |
|PAYMENT |1864.28|21249.0      |19384.72      |0      |1.0         |
|TRANSFER|181.0  |181.0        |0.0           |1      |3.0         |
+--------+-------+-------------+--------------+-------+------------+
only showing top 3 rows



In [45]:
type_encoder = OneHotEncoder(inputCol="type_indexer", outputCol="type_Vector").fit(df)
df = type_encoder.transform(df)

In [46]:
df.show(3,False)

+--------+-------+-------------+--------------+-------+------------+-------------+
|type    |amount |oldbalanceOrg|newbalanceOrig|isFraud|type_indexer|type_Vector  |
+--------+-------+-------------+--------------+-------+------------+-------------+
|PAYMENT |9839.64|170136.0     |160296.36     |0      |1.0         |(4,[1],[1.0])|
|PAYMENT |1864.28|21249.0      |19384.72      |0      |1.0         |(4,[1],[1.0])|
|TRANSFER|181.0  |181.0        |0.0           |1      |3.0         |(4,[3],[1.0])|
+--------+-------+-------------+--------------+-------+------------+-------------+
only showing top 3 rows



In [47]:
df.groupBy('type').count().orderBy('count',ascending=False).show(5,False)

+--------+-------+
|type    |count  |
+--------+-------+
|CASH_OUT|2237500|
|PAYMENT |2151495|
|CASH_IN |1399284|
|TRANSFER|532909 |
|DEBIT   |41432  |
+--------+-------+



In [48]:
df.groupBy('type_indexer').count().orderBy('count',ascending=False).show(5,False)

+------------+-------+
|type_indexer|count  |
+------------+-------+
|0.0         |2237500|
|1.0         |2151495|
|2.0         |1399284|
|3.0         |532909 |
|4.0         |41432  |
+------------+-------+



In [49]:
df_assembler = VectorAssembler(inputCols=['amount', 'oldbalanceOrg', 'newbalanceOrig', 'type_Vector'], outputCol="features")
df = df_assembler.transform(df)

In [50]:
df.select(['features','isFraud']).show(10,False)

+---------------------------------------------+-------+
|features                                     |isFraud|
+---------------------------------------------+-------+
|[9839.64,170136.0,160296.36,0.0,1.0,0.0,0.0] |0      |
|[1864.28,21249.0,19384.72,0.0,1.0,0.0,0.0]   |0      |
|(7,[0,1,6],[181.0,181.0,1.0])                |1      |
|(7,[0,1,3],[181.0,181.0,1.0])                |1      |
|[11668.14,41554.0,29885.86,0.0,1.0,0.0,0.0]  |0      |
|[7817.71,53860.0,46042.29,0.0,1.0,0.0,0.0]   |0      |
|[7107.77,183195.0,176087.23,0.0,1.0,0.0,0.0] |0      |
|[7861.64,176087.23,168225.59,0.0,1.0,0.0,0.0]|0      |
|(7,[0,1,4],[4024.36,2671.0,1.0])             |0      |
|(7,[0,1,2],[5337.77,41720.0,36382.23])       |0      |
+---------------------------------------------+-------+
only showing top 10 rows



In [51]:
df.printSchema()

root
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- type_indexer: double (nullable = false)
 |-- type_Vector: vector (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
model_df=df.select(['features','isFraud'])

In [53]:
from pyspark.ml.classification import LogisticRegression

In [54]:
training_df,test_df=model_df.randomSplit([0.70,0.30],seed=42)

In [55]:
training_df.count()

4454014

In [56]:
training_df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   5702|
|      0|4448312|
+-------+-------+



In [57]:
test_df.count()

1908606

In [58]:
test_df.groupBy('isFraud').count().show()

+-------+-------+
|isFraud|  count|
+-------+-------+
|      1|   2511|
|      0|1906095|
+-------+-------+



In [59]:
log_reg=LogisticRegression(labelCol='isFraud').fit(training_df)

In [60]:
train_results=log_reg.evaluate(training_df).predictions

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [61]:
train_results.filter(train_results['isFraud']==1).filter(train_results['prediction']==1).select(['isFraud','prediction','probability']).show(10,False)

+-------+----------+----------------------------------------+
|isFraud|prediction|probability                             |
+-------+----------+----------------------------------------+
|1      |1.0       |[0.4829243792192139,0.5170756207807861] |
|1      |1.0       |[0.459100466452281,0.540899533547719]   |
|1      |1.0       |[0.44706396477087673,0.5529360352291233]|
|1      |1.0       |[0.4391236776511196,0.5608763223488804] |
|1      |1.0       |[0.42824591076284396,0.571754089237156] |
|1      |1.0       |[0.40347933763355276,0.5965206623664472]|
|1      |1.0       |[0.3943873720634842,0.6056126279365158] |
|1      |1.0       |[0.38120369507917756,0.6187963049208225]|
|1      |1.0       |[0.3556833250833072,0.6443166749166929] |
|1      |1.0       |[0.3294937904532103,0.6705062095467897] |
+-------+----------+----------------------------------------+
only showing top 10 rows



Probability at 0 index is for 0 class and probabilty as 1 index is for 1 class

In [62]:
correct_preds=train_results.filter(train_results['isFraud']==1).filter(train_results['prediction']==1).count()

In [63]:
training_df.filter(training_df['isFraud']==1).count()

5702

In [64]:
#accuracy on training dataset 
float(correct_preds)/(training_df.filter(training_df['isFraud']==1).count())

0.4047702560505086

In [65]:
#Test/validation result
results=log_reg.evaluate(test_df).predictions

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [66]:
results.select(['isFraud','prediction']).show(10,False)

+-------+----------+
|isFraud|prediction|
+-------+----------+
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
|0      |0.0       |
+-------+----------+
only showing top 10 rows



In [67]:
results.printSchema()

root
 |-- features: vector (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [68]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [69]:
#confusion matrix
true_postives = results[(results.isFraud == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.isFraud == 0) & (results.prediction == 0)].count()
false_positives = results[(results.isFraud == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.isFraud == 1) & (results.prediction == 0)].count()

In [70]:
print (true_postives)
print (true_negatives)
print (false_positives)
print (false_negatives)
print(true_postives+true_negatives+false_positives+false_negatives)
print (results.count())

993
1905928
167
1518
1908606
1908606


In [71]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.3954599761051374


In [72]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.8560344827586207


In [73]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9991171567101853


**Random Forest Classifier**

In [78]:
from pyspark.ml.classification import RandomForestClassifier

In [79]:
rf_classifier=RandomForestClassifier(labelCol='isFraud',numTrees=50).fit(training_df)

In [80]:
rf_predictions=rf_classifier.transform(test_df)

In [81]:
rf_predictions.show()

+----------------+-------+--------------------+--------------------+----------+
|        features|isFraud|       rawPrediction|         probability|prediction|
+----------------+-------+--------------------+--------------------+----------+
| (7,[0],[15.46])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[26.95])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[32.27])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[32.58])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[53.39])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[54.24])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[55.78])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[85.02])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[87.92])|      0|[49.9685585924237...|[0.99937117184847...|       0.0|
| (7,[0],[93.59])|      0|[49.9685585924

In [84]:
rf_predictions.columns

['features', 'isFraud', 'rawPrediction', 'probability', 'prediction']

In [82]:
rf_predictions.groupBy('prediction').count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|1907897|
|       1.0|    709|
+----------+-------+



In [85]:
rf_predictions.select(['isFraud','probability','prediction']).show(10,False)

+-------+-----------------------------------------+----------+
|isFraud|probability                              |prediction|
+-------+-----------------------------------------+----------+
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
|0      |[0.9993711718484746,6.288281515254569E-4]|0.0       |
+-------+-----------------------------------------+----------+
only showing top 10 rows



In [86]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [87]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [88]:
rf_accuracy=MulticlassClassificationEvaluator(labelCol='isFraud',metricName='accuracy').evaluate(rf_predictions)

In [89]:
print('The accuracy of RF on test data is {0:.0%}'.format(rf_accuracy))

The accuracy of RF on test data is 100%


In [90]:
print(rf_accuracy)

0.9990495681141105


In [91]:
rf_precision=MulticlassClassificationEvaluator(labelCol='isFraud',metricName='weightedPrecision').evaluate(rf_predictions)

In [92]:
print('The precision rate on test data is {0:.0%}'.format(rf_precision))

The precision rate on test data is 100%


In [93]:
rf_precision

0.9990424728835334

In [94]:
rf_auc=BinaryClassificationEvaluator(labelCol='isFraud').evaluate(rf_predictions)

In [95]:
print(rf_auc)

0.9715284337059189
